In [1]:
import os 

import torch
import torch.nn as nn 
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

from helpers import load_experiment

In [2]:
dm, model = load_experiment("/home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept/outputs/2023-05-03/04-28-27")

Fetching configuration...
Loading datamodule...


[nltk_data] Downloading package wordnet to /home/danis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/home/danis/anaconda3/envs/bottleneck/lib/python3.10/site-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(
100%|██████████| 2700/2700 [00:00<00:00, 9296.29it/s]


Len of vocab:  54
Max len of caption:  16
Index for <pad>: [0]
Loading model


/home/danis/anaconda3/envs/bottleneck/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'criterion' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['criterion'])`.
  rank_zero_warn(


In [3]:
train_loader = dm.train_dataloader()
train_set = train_loader.dataset

In [4]:
# [red, green, blue, square, triangle, circle]

attribute_mapping = {
    0: [1, 0, 0, 1, 0, 0],
    1: [0, 1, 0, 1, 0, 0],
    2: [0, 0, 1, 1, 0, 0],
    3: [1, 0, 0, 0, 1, 0],
    4: [0, 1, 0, 0, 1, 0],
    5: [0, 0, 1, 0, 1, 0],
    6: [1, 0, 0, 0, 0, 1],
    7: [0, 1, 0, 0, 0, 1],
    8: [0, 0, 1, 0, 0, 1],
}

In [5]:
sample = train_set[0]
image = sample["image"]

image = image.unsqueeze(0).cuda()


model(image)["concept_probs"].cpu().detach().numpy()[0]

array([9.6661901e-01, 1.0824219e-03, 9.7497445e-01, 1.6028344e-03,
       4.6688497e-01, 1.1660659e-04, 4.3211441e-04, 2.2364438e-03,
       9.7832537e-01, 5.0284335e-04], dtype=float32)

In [11]:
scores = list()

for feature_id in tqdm(range(10)):

    attribute_values = [[0, 0] for _ in range(6)]

    for i in tqdm(range(len(train_set))):
        sample = train_set[i]
        
        idx = sample["target"].item()
        attributes = np.array(attribute_mapping[idx])

        image = sample["image"]
        image = image.unsqueeze(0).cuda()

        feature = model(image)["concept_probs"].cpu().detach().numpy()[0][feature_id]

        for attr_id, attribute in enumerate(attributes):
            value_on = attribute * feature + (1 - attribute) * (1 - feature)
            attribute_values[attr_id][0] += value_on

            value_off = attribute * (1 - feature) + (1 - attribute) * feature
            attribute_values[attr_id][1] += value_off
    
    attribute_values_max = [max(p) for p in attribute_values]
    attribute_values_max = max(attribute_values_max)
    score = attribute_values_max / len(train_set)

    scores.append(score)

metric = np.array(score).mean()

print(metric)


100%|██████████| 10/10 [06:03<00:00, 36.36s/it]

0.7302641155520837


In [12]:
metric

0.7302641155520837